Author: Miaoyu Yang

NetID: my439

# Features class

In [1]:
class features:
    ''' Class for issues '''

    # keep track of each instance of an issue.
    count = 0 # how many issues have we created?
    all_features = {} # store the issues in a dictionary, aka, a hash table.

    def __init__(self, name):
        ''' This is the constructor for an issue.  It is invoked with the class name and 
        the name of the issue, e.g., issue("abortion")
        We use the Python string method upper() to convert the name to upper case.
        If the issue is already in the dictionary, we ignore this instance. 
        Otherwise, we add it to the dictionary.
        We assign a sequential count to the instance and increment the class count.
        We stick the new issue in dictionary.'''
        self.name = name.upper()
        if self.name not in features.all_features:
            self.count = features.count
            features.count += 1
            features.all_features[self.name] = self

    def __repr__(self):
        ''' Print out a representation that evaluates to this issue.'''
        return f'feature({self.name!r})'

    def __str__(self):
        ''' Return string version of the issue that includes its name and count. '''
        return f"<feature ({self.count}): {self.name}>"
    
    def __eq__(self, other):
        ''' Overload == operator. Two issues must match issue name. '''
        return self.name == other.name 

# Device class

In [2]:
class device:
    count = 0
    devices = {}
    
    def __init__(self, name):
        self.name = name.upper()
        self.device_features = []
        self.df_details = []
        
        if self.name not in device.devices:
            self.count = device.count
            device.count += 1
            device.devices[self.name] = self
            
    def add_device_feature(self, device_feature,df_detail):
        '''Add goals (stances) without duplicates.'''
        if not device_feature in self.device_features:
            self.device_features.append(device_feature)
            self.df_details.append(df_detail)
            
    def __repr__(self):
        ''' Print out agent so that it can evaluate to itself.'''
        return f"device({self.name!r})"
    
    def __str__(self):
        '''Return agent as a string.'''
        return f"<device. name: {self.name} ({self.count})>"
    
    def pp(self):
        '''Pretty print agent information.'''
        result = f"Name:\t{self.name}"
        if self.goals:
            result += f"\nDevice Features:\t{self.device_features}"
        return result
    
    def __eq__(self, other):
        ''' Overload == operator.  Are two agents equal by name and goals? '''
        return self.name == other.name and sorted(self.device_features) == sorted(other.device_features)
    
    def copy(self):
        ''' Clone the agent, including name, and goals. '''
        newdevice = device(self.name)
        newdevice.device_features = self.device_features[:]
        return newdevice
            
    
    
    

# Stance class

In [3]:
class stance:
    ''' Class for importance and side on a given issue.'''
    count = 0
    stances = []

    def __init__(self, featurename, side='pro', importance='A'):
        ''' Constructor for stance().  If the issuename is not already an issue, 
        create a new issue '''
        if not featurename.upper() in features.all_features:
            feature(featurename)
        self.feature = features.all_features[featurename.upper()]
        self.side = side.upper()
        self.importance = importance.upper()
        self.count = stance.count
        stance.count += 1
        stance.stances.append(self)

    def __repr__(self):
        ''' Print out code that evaluates to this stance.'''
        return f'stance({self.feature.name!r}, {self.side!r}, {self.importance!r})'
    
    def __str__(self):
        ''' Return string version of self '''
        return f"<stance ({self.count}): {self.feature.name} [{self.side}:{self.importance}]>"
    
    def __eq__(self, other):
        ''' Overload == operator. Two stances must match issue and side, 
        though not importance. '''
        return self.feature == other.feature and self.side == other.side

    def copy(self):
        ''' Clone a stance.  New stance has same issue, side, and importance. '''
        return stance(self.feature.name, self.side, self.importance)

    def __hash__(self):
        ''' hash() function for stance. 
        Need this for set() to remove duplicates.   
        Note: do not need to include importance.  Match is on issue and side only. '''
        return hash((self.feature.name, self.side))  

    def __lt__(self, other):
        ''' Comparison operator < to allow sorting stances. '''
        return self.feature.name + self.side < other.feature.name + other.side

# Agent Class

An agent has goals - or stances

In [4]:
class agent:
    '''Class for agents who have goals.'''

    count = 0
    agents = []
    mode = 0

    def __init__(self, name):
        ''' Constructor for agent with name.'''
        self.name = name
        self.goals = []
        self.count = agent.count
        agent.count += 1
        agent.agents.append(self)
        self.relationships = []
        self.histories = []


    def __repr__(self):
        ''' Print out agent so that it can evaluate to itself.'''
        return f"agent({self.name!r})"

    def __str__(self):
        '''Return agent as a string.'''
        return f"<agent. name: {self.name} ({self.count})>"

    def add_goal(self, goal):
        '''Add goals (stances) without duplicates.'''
        if not goal in self.goals:
            self.goals.append(goal)
            
    def add_relationship(self,another):
        if another not in self.relationships:
            self.relationships.append(another)
        
    def add_history(self,history):
        if history not in self.histories:
            self.histories.append(history)

    def pp(self):
        '''Pretty print agent information.'''
        result = f"Name:\t{self.name}"
        if self.goals:
            result += f"\nGoals:\t{self.goals}"
        return result


    def __eq__(self, other):
        ''' Overload == operator.  Are two agents equal by name and goals? '''
        return self.name == other.name and sorted(self.goals) == sorted(other.goals) 

    def copy(self):
        ''' Clone the agent, including name, and goals. '''
        newagent = agent(self.name)
        newagent.goals = self.goals[:]
        return newagent

In [5]:
def importance_adjuster(orig, num):
    result = chr(ord(orig) - num)
    if result < "A":
        result = "A"
    elif result > "D":
        result = "D"
    return result


def stance_converter(agent_stance):
    #translate the user's explicit willing to well defined computer related willing
    name = agent_stance.feature.name
    side  = agent_stance.side
    importance = agent_stance.importance
    
    #print(name)
    result = []
    if  "PLAY GAME" in name:
        result.append(stance("High GPU SPEED",side,importance_adjuster(importance,0)))
        result.append(stance("high resolution screen",side,importance_adjuster(importance,1)))
        result.append(stance("large memory",side,importance_adjuster(importance,1)))
        result.append(stance("High speed Internet connection",side,importance_adjuster(importance,0)))
        result.append(stance("high storage",side,importance_adjuster(importance,1)))
        result.append(stance("Windows OS",side,importance_adjuster(importance,4)))
        result.append(stance("High GPU speed",side,importance_adjuster(importance,1)))
    if "TRAVEL" in name:
        result.append(stance("lightweight",side,importance_adjuster(importance,2)))
        result.append(stance("high resolution screen",side,importance_adjuster(importance,-1)))
        result.append(stance("high storage",side,importance_adjuster(importance,-1)))
        result.append(stance("stable",side,importance_adjuster(importance,1)))
        result.append(stance("Big Battery",side,importance_adjuster(importance,1)))
    if "GOOD LOOKING" in name:
        result.append(stance("RGB light",side,importance_adjuster(importance,1)))
        result.append(stance("GOOD Design",side,importance_adjuster(importance,1)))
        result.append(stance("Metal Shell",side,importance_adjuster(importance,0)))
        result.append(stance("Famous brand",side,importance_adjuster(importance,0)))
    if "OFFICE USAGE" in name:
        result.append(stance("Comfortable keyboard",side,importance_adjuster(importance,1)))
        result.append(stance("high resolution screen",side,importance_adjuster(importance,1)))
        result.append(stance("Linux OS",side,importance_adjuster(importance,-2)))
        result.append(stance("stable",side,importance_adjuster(importance,1)))
    if "ECONOMY" in name:
        result.append(stance("low price",side,importance_adjuster(importance,1)))
        result.append(stance("stable",side,importance_adjuster(importance,1)))
        
    return result


def feature_stance_generator(feature,side,importance="A"):
    return stance(feature.name,side,importance)

def importance_compare(i1,i2):
    return (ord(i1)-ord(i2))

importance_compare('A',"D")

-3

In [6]:
from copy import deepcopy

feature_list = ["Famous brand", #0
                "High GPU SPEED", #1
                "high resolution screen",#2
                "large memory", #3
                "High speed Internet connection",#4
                "high storage", #5
                "Windows OS",#6
                "High GPU speed",#7
                "lightweight",#8
                "high storage",#9
                "stable",#10
                "Big Battery",#11
                "RGB light",#12
                "GOOD Design",#13
                "Metal Shell",#14
                "Comfortable keyboard",#15
                "Mac OS",#16
                "Linux OS",#17
                "low price",#18
                "play game",#19
                "travel",#20
                "good looking",#21
                "office usage",#22
                "economy"]

for f in feature_list:
    features(f)
    
product_detail = {}
for i in features.all_features:
    product_detail[i] = (None,None)


In [7]:
side_info = {0:'CON',1:'PRO'}
importance_info = {1:"A",2:"B",3:"C",4:"D"}
product_basic = {
    "MACBOOK PRO":[(0,1,1),(1,1,2),(2,1,1),(3,1,2),(4,1,2),(5,1,2),(6,0,1),(8,1,1),(14,1,1),(16,1,1),(11,0,3),(13,1,1),(18,0,1)],
    "MACBOOK AIR":[(0,1,1),(1,1,3),(2,1,3),(3,1,2),(4,1,2),(5,1,2),(6,0,1),(8,1,1),(14,1,1),(16,1,1),(11,0,3),(13,1,1),(18,0,2)],
    "RAZER BLADE 14":[(0,1,1),(1,1,1),(2,1,1),(3,1,1),(4,1,1),(5,1,1),(6,1,1),(7,1,1),(12,1,1),(13,1,1),(14,0,1),(15,1,1),(18,0,1)],
    "HP ENVY 15.6":[(0,1,1),(1,1,2),(2,1,2),(3,1,2),(4,1,3),(5,1,1),(6,1,1),(7,1,4),(8,1,2),(10,1,2),(15,1,3),(18,1,3)],
    "MICROSOFT SURFACE LAPTOP STUDIO":[(0,1,1),(1,1,1),(2,1,1),(3,1,1),(4,1,1),(5,1,1),(6,1,1),(7,1,1),(8,1,1),(9,1,1),(10,1,1),(15,1,4),(18,0,2)],
    "DELL XPS 15.6":[(0,1,1),(1,1,2),(2,1,2),(3,1,2),(4,1,3),(5,1,1),(6,1,1),(7,1,4),(8,1,2),(10,1,2),(15,1,3),(18,1,3)],
    "LG GRAM 17":[(0,1,1),(1,1,3),(2,1,3),(3,1,2),(4,1,3),(5,1,1),(6,1,1),(7,1,4),(8,1,2),(10,1,2),(15,1,4),(18,1,2)],
    "DELL INSPIRON 7000":[(0,1,1),(1,1,3),(2,1,3),(3,1,3),(4,1,3),(5,1,3),(6,1,1),(7,1,4),(8,1,2),(10,1,2),(15,1,4),(18,1,1)],
    "ASUS ROG ZEPHYRUS G14":[(0,1,1),(1,1,1),(2,1,2),(3,1,1),(4,1,1),(5,1,1),(6,1,1),(7,1,1),(12,1,1),(13,1,1),(14,0,1),(15,1,1),(18,0,2)],
    "ACER PREDATOR TRITON 500":[(0,1,1),(1,1,2),(2,1,2),(3,1,2),(4,1,1),(5,1,2),(6,1,1),(7,1,2),(8,0,2),(9,1,2),(12,1,1),(13,1,1),(14,1,1),(15,1,1),(18,0,2)],
    "HP 15.6":[(0,1,1),(1,1,3),(2,1,3),(3,1,3),(4,1,3),(5,1,4),(6,1,1),(7,0,2),(8,1,3),(9,1,4),(10,1,3),(15,1,4),(18,1,1)],
    "LENOVO THINKPAD X1 YOGA":[(0,1,2),(1,1,2),(2,1,2),(3,1,3),(4,1,1),(5,1,4),(6,1,1),(7,1,4),(8,1,2),(9,1,3),(10,1,2),(15,1,4),(18,0,2)],
    "HP ZBOOK FIREFLY G8":[(0,1,1),(1,1,2),(2,1,2),(3,1,3),(4,1,3),(5,1,3),(6,1,1),(7,1,2),(8,1,2),(9,1,3),(10,1,2),(15,1,4),(18,1,3)],
    "LENOVO LEGION SLIM":[(0,1,2),(1,1,2),(2,1,2),(3,1,2),(4,1,1),(5,1,2),(6,1,1),(7,1,2),(8,0,2),(9,1,2),(12,1,1),(13,1,1),(14,0,1),(15,1,1),(18,1,2)],
    "SAMSUNG GALAXY ODYSSEY 15.6":[(0,1,1),(1,1,2),(2,1,2),(3,1,2),(4,1,3),(5,1,1),(6,1,1),(7,1,4),(8,1,2),(9,1,2),(10,1,2),(14,1,1),(15,1,3),(18,1,3)],
    "MICROSOFT SURFACE BOOK 3":[(0,1,1),(1,1,2),(2,1,2),(3,1,1),(4,1,3),(5,1,1),(6,1,1),(7,1,4),(8,1,2),(10,1,2),(13,1,2),(15,1,3),(18,1,3)],
    "RAZER BOOK 13.4":[(0,1,1),(1,1,2),(2,1,2),(3,1,3),(4,1,3),(5,1,3),(6,1,1),(7,1,3),(8,1,2),(10,1,2),(13,1,2),(15,1,3),(18,1,2)],
    "ASUS EXPERTBOOK B5":[(0,1,1),(1,1,2),(2,1,2),(3,1,2),(4,1,3),(5,1,3),(6,1,1),(7,1,3),(8,1,2),(9,1,2),(10,1,2),(13,1,2),(15,1,3),(18,1,2)],
    "ASUS ZENBOOK DUO 14":[(0,1,1),(1,1,3),(2,1,3),(3,1,2),(4,1,3),(5,1,3),(6,1,1),(7,0,4),(8,1,2),(9,1,2),(10,1,2),(13,1,2),(15,1,3),(18,1,2)],
    "HP SPECTRE X360":[(0,1,1),(1,1,3),(2,1,3),(3,1,3),(4,1,3),(5,1,3),(6,1,1),(7,0,4),(8,1,2),(9,1,3),(10,1,2),(13,1,4),(15,1,3),(18,1,1)]
}
product_info = {}

for i,j in product_basic.items():
    product_info[i] = deepcopy(product_detail)
    for k in j:
        #print(k)
        product_info[i][ feature_list[k[0]].upper() ] = ( side_info[k[1]] ,importance_info[k[2]] )
print(len(product_info))
        

20


In [8]:
device_info = []
for i in product_info:
    d = device(i)
    for f in features.all_features:
        atd = product_info[i][f]
        if (atd[0] is not None) and (atd[1] is not None):
            d.add_device_feature(features.all_features[f],atd)
    device_info.append(d)

In [9]:
def likes(agent, device):
    
    print("="*79)
    proresult = []
    conresult = []
    rejectresult = []
    real_goals = []
    
    score = 0
    
    print('The requirements of the the agent are: ')
    for g in agent.goals:
        print(g.feature.name)
        real_g = stance_converter(g)
        real_goals.extend(real_g)
    
    print("But those requirements are not directly realated to computer recommendation.")
    print("In fact, after conversion, the agent's exact requirements are the following: ")
    print("The device we are considering now is: ")
    print(device.name)
    for g in real_goals:
        print(g)
        for df,atd in zip(device.device_features,device.df_details):
            s = feature_stance_generator(df,atd[0],atd[1])
            if g.feature == s.feature:
                if g.side == s.side:
                    one_s = importance_compare(g.importance,s.importance)
                    score += one_s
                    if one_s < 0:
                        conresult.append(s)
                    else:
                        proresult.append(s)
                else:
                    rejectresult.append(s)
    
    
    answer = len(proresult) - len(conresult)
    """
    print(proresult)
    print(conresult)
    print(rejectresult)
    """
    print("After consideration, We believe the agant:")
    if len(rejectresult) > 0:
        print("Will not like this device. Since this device have feature that the agent cannot accept at all.")
        print("Those feature stance are: ")
        print(rejectresult)
        return (False,score)
    if answer >= 0:
        print("Will like this device. Since there are more prefered features of the device than unprefered features.")
        print("Those prefered feature stance are:")
        print(proresult)
        print("Those unprefered feature stance are:")
        print(conresult)
        return (True,score)
    else:
        print("Will not like this device. Since there are more unprefered features of the device than prefered features.")
        print("Those prefered feature stance are:")
        print(proresult)
        print("Those unprefered feature stance are:")
        print(conresult)
        return (False,score)
    
    
    
    
    """
    if proresult and conresult:
        return ("both", proresult, conresult,rejectresult)
    if proresult:
        return (True, proresult)
    if conresult:
        return (False, conresult)
    else:
        return False
    """

In [10]:
def prefers(agent, list_of_devices):
    print("For now, there are some tentative devices that are available for the agent.")
    print("Those devices are: ")
    
    for device in list_of_devices:
        print(device.name)
    result = []
    for device in list_of_devices:
        r = likes(agent,device)
        if r[0] == True:
            result.append(device)
            
    
    print("="*79)        
    print("After considering of the those devices, the following might be welcomed for the agent:")
    for device in result:
        print(device.name)
    return result

In [11]:
def recommend(agent):
    print("The recommendation precess is based on the relationship and device holding history of those related agent.")
    tentative_devices = []
    for a in agent.relationships:
        for d in a.histories:
            if d not in tentative_devices:
                tentative_devices.append(d)
    print("After collecting all the related agent's device holding history.")
    
    return prefers(agent, tentative_devices)


In [12]:
s1 = stance("GOOD LOOKING","Pro","C")
s2 = stance("PLAY GAME","Pro","C")
s3 = stance("OFFICE USAGE","Pro","A")
s4 = stance("ECONOMY","Pro","A")
s5 = stance("TRAVEL","Pro","A")

a1 = agent('James')
a2 = agent("Tom")
a3 = agent("Peter")
a4 = agent("Sam")
a5 = agent("Andrew")

a1.add_goal(s2)
a1.add_relationship(a2)
a1.add_relationship(a3)
a1.add_history(device_info[0])
a1.add_history(device_info[1])

a2.add_history(device_info[3])
a3.add_history(device_info[4])

a3.add_goal(s3)

likes(a1,device_info[-1])
#prefers(a1,device_info)


The requirements of the the agent are: 
PLAY GAME
But those requirements are not directly realated to computer recommendation.
In fact, after conversion, the agent's exact requirements are the following: 
The device we are considering now is: 
HP SPECTRE X360
<stance (5): HIGH GPU SPEED [PRO:C]>
<stance (6): HIGH RESOLUTION SCREEN [PRO:B]>
<stance (7): LARGE MEMORY [PRO:B]>
<stance (8): HIGH SPEED INTERNET CONNECTION [PRO:C]>
<stance (9): HIGH STORAGE [PRO:B]>
<stance (10): WINDOWS OS [PRO:A]>
<stance (11): HIGH GPU SPEED [PRO:B]>
After consideration, We believe the agant:
Will not like this device. Since this device have feature that the agent cannot accept at all.
Those feature stance are: 
[stance('HIGH GPU SPEED', 'CON', 'D'), stance('HIGH GPU SPEED', 'CON', 'D')]


(False, -3)

In [13]:
prefers(a3,device_info)

For now, there are some tentative devices that are available for the agent.
Those devices are: 
MACBOOK PRO
MACBOOK AIR
RAZER BLADE 14
HP ENVY 15.6
MICROSOFT SURFACE LAPTOP STUDIO
DELL XPS 15.6
LG GRAM 17
DELL INSPIRON 7000
ASUS ROG ZEPHYRUS G14
ACER PREDATOR TRITON 500
HP 15.6
LENOVO THINKPAD X1 YOGA
HP ZBOOK FIREFLY G8
LENOVO LEGION SLIM
SAMSUNG GALAXY ODYSSEY 15.6
MICROSOFT SURFACE BOOK 3
RAZER BOOK 13.4
ASUS EXPERTBOOK B5
ASUS ZENBOOK DUO 14
HP SPECTRE X360
The requirements of the the agent are: 
OFFICE USAGE
But those requirements are not directly realated to computer recommendation.
In fact, after conversion, the agent's exact requirements are the following: 
The device we are considering now is: 
MACBOOK PRO
<stance (96): COMFORTABLE KEYBOARD [PRO:A]>
<stance (97): HIGH RESOLUTION SCREEN [PRO:A]>
<stance (98): LINUX OS [PRO:C]>
<stance (99): STABLE [PRO:A]>
After consideration, We believe the agant:
Will like this device. Since there are more prefered features of the device than

[device('MACBOOK PRO'),
 device('RAZER BLADE 14'),
 device('MICROSOFT SURFACE LAPTOP STUDIO'),
 device('ASUS ROG ZEPHYRUS G14'),
 device('ACER PREDATOR TRITON 500'),
 device('LENOVO LEGION SLIM')]

In [14]:
recommend(a1)

The recommendation precess is based on the relationship and device holding history of those related agent.
After collecting all the related agent's device holding history.
For now, there are some tentative devices that are available for the agent.
Those devices are: 
HP ENVY 15.6
MICROSOFT SURFACE LAPTOP STUDIO
The requirements of the the agent are: 
PLAY GAME
But those requirements are not directly realated to computer recommendation.
In fact, after conversion, the agent's exact requirements are the following: 
The device we are considering now is: 
HP ENVY 15.6
<stance (1120): HIGH GPU SPEED [PRO:C]>
<stance (1121): HIGH RESOLUTION SCREEN [PRO:B]>
<stance (1122): LARGE MEMORY [PRO:B]>
<stance (1123): HIGH SPEED INTERNET CONNECTION [PRO:C]>
<stance (1124): HIGH STORAGE [PRO:B]>
<stance (1125): WINDOWS OS [PRO:A]>
<stance (1126): HIGH GPU SPEED [PRO:B]>
After consideration, We believe the agant:
Will like this device. Since there are more prefered features of the device than unprefered

[device('HP ENVY 15.6'), device('MICROSOFT SURFACE LAPTOP STUDIO')]